In [2]:
def draw_utility (utility):
  for i in range(3):
    for j in range(3):
      print("{:.2f}\t".format(utility[i][j]), end = "")
    print()

In [3]:
def draw_policy(policy):
  s = [[str(e) for e in row] for row in policy]
  lens = [max(map(len, col)) for col in zip(*s)]
  fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
  table = [fmt.format(*row) for row in s]
  print('\n'.join(table))


# Ideas


we use value iteration and policy iteration to get the optimal policy

**value iteration** starts from utality = 0 then start to update the values using bellman equation until it convergs and then stop

**policy iteration** start with random utality and then use bellman equtaion to update utality until it convergs then stop

# Algorithms

## 1- Value Iteration



1.   Go through each square on the grid
2.   Update the utility using Bellman equation
3.   Keep iterate and update the utility while the difference between new value and old value < x 
4.   Return the final policy 




In [4]:
def summation2(action, reward, utility, tansition_dic, discount, i, j) :
  n = i - 1 if i > 0 else  0
  s = i + 1 if i < 2 else  2
  e = j + 1 if j < 2 else  2
  w = j - 1 if j > 0 else  0
  ans = tansition_dic[action][0]*(reward[n][j] + discount * utility[n][j]) +\
        tansition_dic[action][1]*(reward[s][j] + discount * utility[s][j]) +\
        tansition_dic[action][2]*(reward[i][e] + discount * utility[i][e]) +\
        tansition_dic[action][3]*(reward[i][w] + discount * utility[i][w]) 

  return ans

In [38]:
from collections import Counter
import time
import copy 

def value_iteration(r, threshold):
  utility = [[0,0,0],[0,0,0],[0, 0, 0]]  # utility of the board starts with 0s

  reward = [[r, -1, 10], [-1, -1 , -1], [-1, -1, -1]] # reward of each square in the grid
  gama = 0.99     # discount factor 
  actions = ['n', 's', 'e', 'w']  # available actions 
  
  tansition_dic = {               # actions probabilities 
      'n' : [0.8, 0 ,0.1, 0.1],
      's' : [0, 0.8, 0.1, 0.1],
      'e' : [0.1, 0.1, 0.8, 0],
      'w' : [0.1, 0.1, 0, 0.8],
  }

  policy =[[r , '', '+10'], ['', '' ,''], ['', '', '']]  # the policy of the agent 
  
  thres = 100
  counter = 0
  while True:        # for loop only for testing, will be while later
    if(thres < threshold):            # if all Vi+1 - Vi < threshold then we reach our limit break
      break
    if counter == 0 :
      utility = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
      counter +=1
      continue

    counter +=1
    
    thres = 0
    v = copy.deepcopy(utility)         # save the old  values of utility to compare it with the new vlaues
    delta = 0
    
    # go through all the squresin the grid
    for i in range(3):
      for j in range(3):
        if (i==0) and (j==0 or j==2) : continue
        prev = summation2('n', reward, v, tansition_dic, gama, i, j) 
        prevAction = 'n'
        for action in actions:      #go through all actions for each square on the grid
          utility[i][j] = summation2(action, reward, v, tansition_dic, gama, i, j)    #get the utility of each action
          policy[i][j] = action
          
          if utility[i][j] < prev :
            utility[i][j] = prev
            policy[i][j] = prevAction
          else :
            prev = utility[i][j]
            prevAction = action

    for i in range(3):
      for j in range(3):
        thres = max(thres, abs(utility[i][j] - v[i][j]))
  print("number of iterations : " + str(counter))

  #draw_utility(utility)
  return policy

## 2- Policy Iteration 

1.  start with random initial policy
2.  evaluate the policy to get the initial utility using bellman equation 
3.  update the policy according to the new utility 
4.  return the final policy 

In [6]:
import random
def get_random_pos():
  i = random.randint(0,2)
  j = random.randint(0,2)
  return i, j

In [7]:
def get_random_policy(r):
  moves = ['n', 's', 'e','w']
  policy = [[r, 'b', '+10'],['a', 'b', 'c'],['a', 'b', 'c']]
  for i in range (0, 3):
    for j in range (0, 3):
      if (i==0) and (j==0 or j==2) : continue
      policy[i][j] = moves[random.randint(0,3)]
 
  return policy

In [8]:
# import numpy as np
def policy_evaluation(reward, policy, utility, discount_factor):
  threshold = 0.001
  converge = False
  tansition_dic = {
      'n' : [0.8, 0 ,0.1, 0.1],
      's' : [0, 0.8, 0.1, 0.1],
      'e' : [0.1, 0.1, 0.8, 0],
      'w' : [0.1, 0.1, 0, 0.8],
  }
  discount = 0.99
  policy_evaluation_counter = 0
  while not converge:
    policy_evaluation_counter+=1
    delta = 0
    prev_utility = [row[:] for row in utility]
    for i in range(0, 3):
      for j in range(0, 3):
        if (i==0) and (j==0 or j==2) : continue
        temp = utility[i][j]
        utility[i][j] = summation2(policy[i][j], reward, prev_utility, tansition_dic, discount, i, j)
        delta = max(delta, abs(temp - utility[i][j]))

    if delta < threshold:
      converge = True
  return utility

In [9]:
def get_max_action(utility, reward, discount,  current_i, current_j) :
  actions = ['n', 's', 'e', 'w']
  tansition_dic = {
      'n' : [0.8, 0 ,0.1, 0.1],
      's' : [0, 0.8, 0.1, 0.1],
      'e' : [0.1, 0.1, 0.8, 0],
      'w' : [0.1, 0.1, 0, 0.8],
  }
  max_action = 'n'
  max_value = summation2(max_action, reward, utility, tansition_dic, discount, current_i, current_j)
  for action in actions :
    value = summation2(action, reward, utility, tansition_dic, discount, current_i, current_j)
    if value > max_value :
      max_value = value
      max_action = action

  return max_action

In [10]:
def policy_improvement(policy, utility, reward, discount):

  change  = False
  for i in range(3) :
    for j in range(3) :
      if (i==0) and (j==0 or j==2) : continue
      temp = policy[i][j]
      policy[i][j] = get_max_action(utility, reward, discount, i, j)
      if temp != policy[i][j] :
        change = True
  return policy , change

In [37]:
def policy_iteration(r):
  policy = get_random_policy(r)
  discount_factor = 0.99
  utility = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
  reward = [[r, -1, 10], [-1, -1 , -1], [-1, -1, -1]]
  stable = False

  iterations_counter = 0
  while not stable:
    if not iterations_counter :
      utility = [[r, 0, 10], [0, 0, 0], [0, 0, 0]]
      iterations_counter += 1
      continue
    iterations_counter += 1
    utility = policy_evaluation(reward, policy, utility, discount_factor)
    policy, change = policy_improvement(policy, utility, reward, discount_factor)
    if not change:
      stable = True
      
  print("number of iterations is : " + str(iterations_counter))
  #draw_utility(utility)
  return policy 

# output Policies 

### r = 100

Value Iteration

In [39]:
policy = value_iteration(100, 0.000001)
print()
draw_policy(policy)

number of iterations : 33

100	w	+10
n  	w	s  
n  	w	w  


Policy Itreation 

In [40]:
policy = policy_iteration(100)
print()
draw_policy(policy)

number of iterations is : 4

100	w	+10
n  	w	s  
n  	w	w  


**the optimal policy tends to get the highest reward, so it escape from the smalles reward r = 10 by going south and move to the left side of the grid to get the highest reward r = 100**

### r = 3

Valeu Iteration

In [41]:
policy = value_iteration(3, 0.000001)
print()
draw_policy(policy)

number of iterations : 28

3	e	+10
e	e	n  
e	e	n  


Policy Iteration

In [42]:
policy = policy_iteration(3)
print()
draw_policy(policy)

number of iterations is : 4

3	e	+10
e	e	n  
e	e	n  


**the policy tends to get the highest reward so it moves to the right side with the highest reward r = 10**

### r = 0

Value Iteration

In [43]:
policy = value_iteration(0, 0.000001)
print()
draw_policy(policy)

number of iterations : 28

0	e	+10
e	e	n  
e	e	n  


Policy Iteration

In [44]:
policy = policy_iteration(0)
print()
draw_policy(policy)

number of iterations is : 4

0	e	+10
e	e	n  
e	e	n  


**the policy tends to get the highest reward so it moves to the right side with the highest reward r = 10**

### r = -3

Value Iteration

In [45]:
policy = value_iteration(-3, 0.000001)
print()
draw_policy(policy)

number of iterations : 28

-3	e	+10
e 	e	n  
e 	e	n  


Policy Iteration

In [46]:
policy = policy_iteration(-3)
print()
draw_policy(policy)

number of iterations is : 3

-3	e	+10
e 	e	n  
e 	e	n  


**the policy tends to get the highest reward so it moves to the right side with the highest reward r = 10**

### r = -300

Value Iteration

In [47]:
policy = value_iteration(-300, 0.000001)
print()
draw_policy(policy)

number of iterations : 30

-300	e	+10
s   	e	n  
e   	e	n  


Policy Iteartion 

In [48]:
policy = policy_iteration(-300)
print()
draw_policy(policy)

number of iterations is : 4

-300	e	+10
s   	e	n  
e   	e	n  


**policy tends to excape form the small value and move to the highest value, So It escapes from this exit square with r = -300 as it makes much loss by moving south and then direct to the other exit with the highest reward**